In [2]:
from fastai import *        # Quick access to most common functionality
from fastai.text import *   # Quick access to NLP functionality

In [5]:
import fastai
print(fastai.__version__)

1.0.18


In [3]:
train_df = pd.read_csv("data/train.csv")
train_df.head(1)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,a50d1fb6c9818058,Listen Ho! \n\nYOU'RE A HO!,0,0,0,0,0,0


In [4]:
train_df = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'comment_text']]
train_df = train_df.iloc[:400, :]
train_df.head(1)

,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text
0,0,0,0,0,0,0,Listen Ho! \n\nYOU'RE A HO!


In [5]:
train_df.shape

(400, 7)

In [6]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train_df, test_size = 0.2)
len(train), len(valid)

(320, 80)

In [7]:
torch.cuda.set_device(1)

In [8]:
path = Path('ULMFiT/toxic')
path.mkdir(exist_ok = True)

In [9]:
train.to_csv(path/"train.csv", header=False, index=False)
valid.to_csv(path/"valid.csv", header=False, index=False)

In [10]:
valid = pd.read_csv(path/"valid.csv", header=None) # to test that there are no headers
valid.head(1)

,0,1,2,3,4,5,6
0,0,0,0,0,0,0,"""\n\nOK let's agree on one thing first: Includ..."


In [11]:
data_lm = TextLMDataBunch.from_csv(path=path)

Numericalizing valid.


In [29]:
data_clas = TextClasDataBunch.from_csv(path=path, vocab=data_lm.train_ds.vocab, bs=32, n_labels=6)

In [30]:
learn = RNNLearner.language_model(data_lm, pretrained_model=URLs.WT103, drop_mult=0.5)

In [31]:
learn.fit_one_cycle(cyc_len=1, max_lr=1e-3)

Total time: 00:02
epoch  train_loss  valid_loss  accuracy
1      5.507805    4.786237    0.199219  (00:02)



In [32]:
learn.save_encoder('lm_encoder')

In [33]:
learn = RNNLearner.classifier(data_clas, drop_mult=0.5)
learn.metrics = []
learn.load_encoder('lm_encoder')
learn.fit_one_cycle(cyc_len=1, max_lr=1e-3)

epoch,train_loss,valid_loss


ValueError: Target size (torch.Size([16])) must be the same as input size (torch.Size([16, 2]))